## **Predicting Seasonal Flu Vaccination Uptake**

## Final Project Submission

* Student name: Colleta Kiilu
* Student pace:  **part time** 
* Scheduled project submission date/time: 23/12/2024
* Instructor name: Samuel Karu
* Blog post URL: https://github.com/Collz472/Phase_3_Project_Colleta.git

# **Business Understanding**


## **Project Background**

Vaccination is a critical public health intervention for controlling and preventing the spread of infectious diseases. 
By providing immunization at an individual level and fostering herd immunity within communities, vaccines play a very important role in safeguarding global health. 
In 2009, the H1N1 influenza virus, commonly known as "swine flu," caused a global pandemic, resulting in an estimated 151,000 to 575,000 deaths worldwide in its first year. 
To combat this pandemic, an H1N1 vaccine was made available in October 2009.

The United States National 2009 H1N1 Flu Survey, conducted in late 2009 and early 2010, collected data on vaccination uptake for both H1N1 and seasonal flu. 
The survey explored respondents’ vaccination status alongside information about their socioeconomic and demographic backgrounds, health behaviors, and opinions on vaccine efficacy and illness risk. 
This project therefore seeks to guide furture public health strategies by providing valuable insights into the factors influencing the vaccination patterns through a thorough analysis of the dataset provided.

## **Problem Statement**

Understanding the factors that influence vaccination decisions is essential for designing effective public health campaigns. 
Despite the availability of vaccines for the seasonal flu, uptake rates varied significantly among different population groups. 
The challenge lies in identifying and analyzing the individual and social factors that determine vaccine adoption, which is critical for improving vaccination rates and achieving herd immunity during pandemics.

## **General Objective**

To investigate the factors influencing individuals’ decisions to receive the seasonal flu vaccine using data on their backgrounds, health behaviors, and opinions, with the goal of informing public health strategies to enhance vaccine uptake.

## **Specific Objectives**

1. To analyze the relationship between demographic factors, opinions on vaccine effectiveness, and health behaviors in influencing vaccination status.

2. To identify key socioeconomic and behavioral barriers to accessing vaccines.

3. To develop predictive models for determining the likelihood of individuals receiving H1N1 and seasonal flu vaccines.

## **Research Questions**

1. What are the key demographic, behavioral, and opinion-based factors influencing vaccination decisions?

2. What socioeconomic barriers hinder vaccine accessibility and uptake?

3. How accurately can predictive models identify individuals likely to receive or forgo vaccination?

# **Data Understanding**



The National 2009 H1N1 Flu Survey (NHFS) was conducted between October 2009 and June 2010 to assess the uptake of H1N1 and seasonal flu vaccines in the United States. The survey was designed as a phone-based data collection effort, targeting a representative sample of the U.S. population. Key data collected included:

1. **Vaccination Status**: Whether respondents received the H1N1 vaccine, the seasonal flu vaccine, both, or neither.

2. **Demographic Information**: Age, gender, race/ethnicity, income level, and education.

3. **Health Behaviors**: Preventative practices such as handwashing and mask usage.

4. **Opinions and Perceptions**: Views on vaccine efficacy, risks of illness, and general attitudes toward vaccination.

5. **Socioeconomic Context**: Factors that might influence access to healthcare services, such as insurance status and geographic location.

The data is composed of approximately 26,000 instances of individual data and vaccine decision information.

### **Data Source** 
The data used for this project comes from the National 2009 H1N1 Flu Survey (NHFS) and was provided courtesy of the United States National Center for Health Statistics. 

[U.S. Department of Health and Human Services (DHHS). National Center for Health Statistics. The National 2009 H1N1 Flu Survey. Hyattsville, MD: Centers for Disease Control and Prevention, 2012.](https://www.drivendata.org/competitions/66/flu-shot-learning/data/)

The data labels and features are as described in ths [link.](https://www.drivendata.org/competitions/66/flu-shot-learning/page/211/#labels)


In [4]:
# import Libraries

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt



from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier



import warnings
warnings.filterwarnings('ignore')

## **Import the data sets**

In [5]:
#import datasets

target = pd.read_csv("training_set_labels.csv")
train = pd.read_csv("training_set_features.csv")
test = pd.read_csv("test_set_features.csv")



# Display the first few rows of each DataFrame to ensure data loading was successful

print("\nTarget Labels:")
print(target.head())

print("Training Data:")
print(train.head())

print("\nTest Data:")
print(test.head())


Target Labels:
   respondent_id  h1n1_vaccine  seasonal_vaccine
0              0             0                 0
1              1             0                 1
2              2             0                 0
3              3             0                 1
4              4             0                 0
Training Data:
   respondent_id  h1n1_concern  h1n1_knowledge  behavioral_antiviral_meds  \
0              0           1.0             0.0                        0.0   
1              1           3.0             2.0                        0.0   
2              2           1.0             1.0                        0.0   
3              3           1.0             1.0                        0.0   
4              4           2.0             1.0                        0.0   

   behavioral_avoidance  behavioral_face_mask  behavioral_wash_hands  \
0                   0.0                   0.0                    0.0   
1                   1.0                   0.0                    1.0

## *Data Exploration and data Cleaning**

In [6]:
 #Examine Data Shape and Size

print("Target Shape:", target.shape)
print("Train Shape:", train.shape)
print("Test Shape:", test.shape)


Target Shape: (26707, 3)
Train Shape: (26707, 36)
Test Shape: (26708, 36)


**The target  dataset** contains 26,707 rows and 3 columns

**The train feature** dataset contains 26,707 rows and 36 columns.

**The test dataset** contains 26,708 rows and 36 columns

In [7]:
# Print column names and data types of the "target" DataFrame
print("\nTarget Labels Columns:")
print(target.columns)


Target Labels Columns:
Index(['respondent_id', 'h1n1_vaccine', 'seasonal_vaccine'], dtype='object')


In [8]:
# Print column names and data types of the "train" DataFrame
print("\nTrain Labels Columns:")
print(train.columns)


Train Labels Columns:
Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')


In [9]:
# Print column names and data types of the "test" DataFrame
print("\nTest Labels Columns:")
print(test.columns)


Test Labels Columns:
Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation'],
      dtype='object')


### **Remove Unnecessary Columns**

**Drop information for HINI**

There are two potential targets in these datasets: whether the survey respondent received the seasonal flu vaccine, or whether the respondent received the H1N1 flu vaccine. For my minimum viable project, i will choose just one of these potential targets therefore, i will drop all information related to H1N1 flu and maintain the seasonal flu vaccination. The columns are not directly relevant to the current analysis

In [10]:
# Drop columns related to H1N1 flu from the "target" DataFrame
target.drop(columns=["h1n1_vaccine"], axis=1, inplace=True)

# Drop columns related to H1N1 flu from the "train" DataFrame
train.drop(columns=["h1n1_concern","h1n1_knowledge","doctor_recc_h1n1","opinion_h1n1_vacc_effective","opinion_h1n1_risk","opinion_h1n1_sick_from_vacc"], axis=1, inplace=True)

# Drop columns related to H1N1 flu from the "test" DataFrame
test.drop(columns=["h1n1_concern","h1n1_knowledge","doctor_recc_h1n1","opinion_h1n1_vacc_effective","opinion_h1n1_risk","opinion_h1n1_sick_from_vacc"], axis=1, inplace=True)


In [11]:
# Cconfirm if the columns dropped

print("target Shape:", target.shape)
print("train Shape:", train.shape)
print("set Shape:", test.shape)


target Shape: (26707, 2)
train Shape: (26707, 30)
set Shape: (26708, 30)


**The target  dataset** now contains 26,707 rows and 2 columns. 1 columns was successfuly dropped

**The train feature** dataset now contains 26,707 rows and 30columns. 6 columns were successfuly dropped

**The test dataset** now contains 26,708 rows and 30 columns. 6 columns were successfuly dropped

## **Checking for Missing values**

In [12]:
# check the percentage of missing values per column

missing=(train.isnull().sum()/len(train))*100

# show columns with the highest missing percentages
missing = missing.sort_values(ascending=False)

print("Missing values:")
print(missing)

Missing values:
employment_occupation          50.436215
employment_industry            49.912008
health_insurance               45.957989
income_poverty                 16.561201
doctor_recc_seasonal            8.087767
rent_or_own                     7.645936
employment_status               5.477965
marital_status                  5.272026
education                       5.268282
chronic_med_condition           3.635751
child_under_6_months            3.070356
health_worker                   3.010447
opinion_seas_sick_from_vacc     2.010709
opinion_seas_risk               1.924589
opinion_seas_vacc_effective     1.729884
household_adults                0.932340
household_children              0.932340
behavioral_avoidance            0.778822
behavioral_touch_face           0.479275
behavioral_large_gatherings     0.325757
behavioral_outside_home         0.307036
behavioral_antiviral_meds       0.265848
behavioral_wash_hands           0.157262
behavioral_face_mask            0.071142


In [ ]:
# Count missing values and sort in ascending order
missing_values_sorted = train.isnull().sum().sort_values(ascending=False)

print(missing_values_sorted)

employment_occupation          13470
employment_industry            13330
health_insurance               12274
income_poverty                  4423
doctor_recc_seasonal            2160
rent_or_own                     2042
employment_status               1463
marital_status                  1408
education                       1407
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
opinion_seas_sick_from_vacc      537
opinion_seas_risk                514
opinion_seas_vacc_effective      462
household_adults                 249
household_children               249
behavioral_avoidance             208
behavioral_touch_face            128
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_antiviral_meds         71
behavioral_wash_hands             42
behavioral_face_mask              19
age_group                          0
race                               0
sex                                0
h

**Observations**

1. From the above information, the employment_occupation column, employment_industry column and the health_insurance column have the highest number of missing values at 50.4%, 49.9% and 45.96% respectively

2. From the data set, there is a strong relationship between the employement_status cloumn and the employment_occupation and employement_industry columns which have the highest null values. The exected responses in the employement_status column was either; 1) employed, 2) unemployed or 3) not in labour force. It is therfore expected that those who responded 'unemplyed' or 'not in labour force', would leave the the employment_occupation and employement_industry columns blank.

3. Therefore, 10,231 of the null values for the employment_occupation and employemnt_indstry will be viewed as "not applicable" as those are the the respondents who answered "Not in Labor Force" for employment_status. The information will be viewed as not applicable as opposed to respondendt not choosing to respond

4. Similarly, an additional 1,453 of the null values in the employment_occupation and employmet_industry representing all unemployed individuals will be viewed as not applicable. 


## **Handling Missing values** 

**1. Employment Columns** (employment_industry and employment_occupation columns)
- For respondents marked as "Unemployed" in the `employment_status` column, the `employment_industry` will be updated to "not employed".
- For respondents marked as "Unemployed" in the `employment_status` column, the `employment_occupation` will be updated to "not employed".
- For respondents marked as "Not in Labor Force" in the `employment_status` column, the `employment_industry` will be updated to "not employed".
- For respondents marked as "Not in Labor Force" in the `employment_status` column, the `employment_occupation` is also updated to "not employed".  

By labeling these cases as `not employed`, the missing values are now replaced with meaningful information, which can be utilized for further analysis without introducing any biased assumptions.

In [20]:
## if a person is unemployed, change their "employment_industry" to "not_employed"
train.loc[train["employment_status"] == "Unemployed", "employment_industry"] = "not employed"

## if a person is unemployed, change their "employment_occupation" to "not_employed"
train.loc[train["employment_status"] == "Unemployed", "employment_occupation"] = "not employed"

## if a person is not in the labor force, change their "employment_industry" to "not_employed"
train.loc[train["employment_status"] == "Not in Labor Force", "employment_industry"] = "not employed"

## if a person is not in the labor force, change their "employment_occupation" to "not_employed"
train.loc[train["employment_status"] == "Not in Labor Force", "employment_occupation"] = "not employed"

**2. `health_insurance` Column**

- Missing values in the "health_insurance" column are filled with 0, assuming that those with missing values likely do not have health insurance coverage, possibly due to financial constraints associated with a higher poverty index.

In [21]:
#Filling the missing values in health insurance with 0 assumptions is that there is correlation between the poverty index, employemnt status and health cover

train["health_insurance"].fillna(0, inplace=True)

In [22]:
# confimr missing values again 
missing=(train.isnull().sum()/len(train))*100

missing = missing.sort_values(ascending=False)

print("Missing values:")
print(missing)

Missing values:
income_poverty                 16.561201
doctor_recc_seasonal            8.087767
rent_or_own                     7.645936
employment_occupation           6.687385
employment_industry             6.163178
employment_status               5.477965
marital_status                  5.272026
education                       5.268282
chronic_med_condition           3.635751
child_under_6_months            3.070356
health_worker                   3.010447
opinion_seas_sick_from_vacc     2.010709
opinion_seas_risk               1.924589
opinion_seas_vacc_effective     1.729884
household_children              0.932340
household_adults                0.932340
behavioral_avoidance            0.778822
behavioral_touch_face           0.479275
behavioral_large_gatherings     0.325757
behavioral_outside_home         0.307036
behavioral_antiviral_meds       0.265848
behavioral_wash_hands           0.157262
behavioral_face_mask            0.071142
age_group                       0.000000
